In [ ]:
import pandas as pd
import numpy as np
from utils import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
years = ['0304', '0506', '0708', '0910', '1112', '1314', '1516', '1718', '1720']
year_char = 'C'

### 1. Collect demographic information

In [ ]:
type_demo = 'demographic'
df_demo = concat_data_across_years(type_demo, 'DEMO', years, year_char)
# 95872 unique records in total

In [ ]:
# Select the wanted columns. Make changes here if needed in the future.
df_demo = df_demo[['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'INDHHINC', 'DMDEDUC2', 'WTINT2YR',
                       'WTMEC2YR', 'WTINTPRP', 'WTMECPRP', 'years']]
df_demo = df_demo.fillna(0)
df_demo[['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'INDHHINC', 'DMDEDUC2']] = \
    df_demo[['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'INDHHINC', 'DMDEDUC2']].astype(int)

In [ ]:
# Unify the weight columns and make column names readable.
df_demo['weight_interview'] = np.where(df_demo['WTINT2YR'] == -1, df_demo['WTINTPRP'], df_demo['WTINT2YR'])
df_demo['weight_mec'] = np.where(df_demo['WTMEC2YR'] == -1, df_demo['WTMECPRP'], df_demo['WTMEC2YR'])
df_demo.drop(['WTINT2YR','WTMEC2YR', 'WTINTPRP', 'WTMECPRP'], axis=1, inplace=True)
df_demo = df_demo.rename(columns={'RIAGENDR': 'gender', 'RIDAGEYR': 'age', 'RIDRETH1': 'race', 'DMDEDUC2': 'education',
                                  'INDHHINC': 'household_income'})
df_demo = df_demo.set_index('SEQN')

In [ ]:
# Transform ages to age groups
bins = [-1, 10, 20, 30, 40, 50, 60, 100]
labels = ['1', '2', '3', '4', '5', '6', '7']

# Create a new column for age groups
df_demo['age_group'] = pd.cut(df_demo['age'], bins=bins, labels=labels, right=True)
df_demo.drop('age', axis=1, inplace=True)

### 2. Questionnaire Data

In [ ]:
type_questionnaire = 'questionnaire'

#### 2.1. Diet Behaviors & Nutrition

In [ ]:
# For simplicity, I use the acronym of each initials as the variable name, so Diet Behaviors & Nutrition is DBN
df_DBN = concat_data_across_years(type_questionnaire, 'DBQ', years, year_char)

In [ ]:
df_DBN = df_DBN[['SEQN', 'DBD090', 'DBD091', 'DBD197', 'DBQ197', 'DBQ229', 'DBD895', 'DBQ700',
                 'DBD900', 'DBD905', 'DBD910']]
df_DBN = df_DBN.fillna(-1)
df_DBN = df_DBN.astype(int)

In [ ]:
# These 3 columns are the same questions of different field names over the years
df_DBN['meals_outside'] = np.where(df_DBN['DBD090'] == -1, df_DBN['DBD091'], df_DBN['DBD090'])
df_DBN['meals_outside'] = np.where(df_DBN['meals_outside'] == -1, df_DBN['DBD895'], df_DBN['meals_outside'])
# For those who had meals outside more than 21 time a week, we mark it to 22.
# For those who had meals outside less than once a week, we mark it 0.
df_DBN['meals_outside'].loc[df_DBN['meals_outside'] == 5555] = 22
df_DBN['meals_outside'].loc[df_DBN['meals_outside'] == 6666] = 0
df_DBN.drop(['DBD090', 'DBD091', 'DBD895'], axis=1, inplace=True)

In [ ]:
df_DBN['milk_consumption_last_30_days'] = np.where(df_DBN['DBD197'] == -1, df_DBN['DBQ197'], df_DBN['DBD197'])
df_DBN.drop(['DBD197', 'DBQ197'], axis=1, inplace=True)

In [ ]:
df_DBN = df_DBN.rename(columns={'DBQ229': 'regular_milk_drinker', 'DBQ700': 'self_perceived_diet',
                                'DBD900': 'fastfood_weekly', 'DBD905': 'ready_to_eat_food_monthly',
                                'DBD910': 'frozen_meal_monthly', 'meals_outside': 'meals_outside_weekly'})

In [ ]:
# The same code might indicate different things. For weekly data, 6666 is less than once a week.
# For monthly data, 6666 is more than 90 times a month.
df_DBN['fastfood_weekly'].loc[df_DBN['fastfood_weekly'] == 5555] = 22
df_DBN.replace([7777, 9999], -1, inplace=True)
df_DBN.replace([6666], 91, inplace=True)

In [ ]:
df_DBN = df_DBN.set_index('SEQN')
df_DBN

,regular_milk_drinker,self_perceived_diet,fastfood_weekly,ready_to_eat_food_monthly,frozen_meal_monthly,meals_outside_weekly,milk_consumption_last_30_days
SEQN,,,,,,,
21005,-1,-1,-1,-1,-1,0,2
21006,-1,-1,-1,-1,-1,1,3
21007,-1,-1,-1,-1,-1,1,3
21008,-1,-1,-1,-1,-1,1,3
21009,3,-1,-1,-1,-1,8,2
...,...,...,...,...,...,...,...
124818,2,4,4,5,0,4,1
124819,-1,-1,-1,0,0,0,1
124820,-1,-1,1,0,0,2,3


In [ ]:
df_DBN.to_csv('../processed_data/diet_habits.csv')

#### 2.2. Drug Use

In [ ]:
# No specific illicit drug info in 03-04 data. No such table in 17-20 data.
df_DU = concat_data_across_years(type_questionnaire, 'DUQ',
                                 ['0506', '0708', '0910', '1112', '1314', '1516', '1718'], 'D')

In [ ]:
df_DU = df_DU.loc[df_DU['DUQ290'] == 1]
df_DU = df_DU[['SEQN', 'DUQ270U', 'DUQ350U', 'DUQ300', 'DUQ310Q', 'DUQ310U']]
df_DU = df_DU.fillna(-1)
df_DU = df_DU.astype(int)

In [ ]:
"""
# We only care about those who have used heroin at least once.
# If within a year, the user has been using any illicit drugs (heroin, meth, cocaine),
# we identify the user as an active user.
# Otherwise, we consider this user a recovered user.

# To be specific, -1 means missing, 4 means it has been years that a user haven't used a drug.
# If there is a value that is neither -1, nor 4 in any of the three columns, the user is an active user.

# We label the active user as 1 and recovered user as 2.
# And the rest of the users who hasn't even used heroin or other opioid prescription drugs as 0: non-opioid-user
"""
df_DU['active_user'] = np.where((df_DU['DUQ270U'].isin([4, -1]) == False) |
                                 (df_DU['DUQ350U'].isin([4, -1]) == False) |
                                 (df_DU['DUQ310U'].isin([4, -1]) == False), 1, 2)

In [ ]:
df_DU = df_DU.rename(columns={'DUQ300': 'age_first_use_heroin', 'DUQ310U': 'last_time_unit_used_heroin',
                              'DUQ310Q': 'last_time_used_heroin', 'DUQ270U': 'last_time_unit_used_cocaine',
                              'DUQ350U': 'last_time_unit_used_meth'})

In [ ]:
"""
last_time_unit_used_heroin and last_time_used_heroin is a combo feature.
time unit defines whether it's years, months or days we are talking about.
the time defines the exact number of that unit.
For example, if time unit is 4 and time is 30, it means this user last used heroin 30 years ago.
"""
df_DU = df_DU.set_index('SEQN')
df_DU

,last_time_unit_used_cocaine,last_time_unit_used_meth,age_first_use_heroin,last_time_used_heroin,last_time_unit_used_heroin,active_user
SEQN,,,,,,
31452,4,4,19,30,4,2
31551,4,-1,18,31,4,2
31577,4,4,20,32,4,2
31827,4,-1,21,25,4,2
32071,4,4,16,6,4,2
...,...,...,...,...,...,...
102548,-1,-1,21,7,4,2
102605,4,-1,18,20,4,2
102623,-1,-1,-1,-1,-1,2


In [ ]:
# df_drug_age = df_demo.merge(df_DU, how='right', left_on=df_demo.index, right_on=df_DU.index)

#### 2.3.Prescription Medicine

In [ ]:
df_PM = concat_data_across_years(type_questionnaire, 'RXQ_RX', years, year_char)

In [ ]:
df_PM = df_PM[df_PM['RXDUSE']==1]

In [ ]:
df_PM['SEQN'] = df_PM['SEQN'].astype(int).astype(str)
df_PM[['SEQN', 'RXDDRUG', 'RXDDRGID', 'RXDDAYS']].to_csv('../processed_data/user_prescription_medicine.csv', index=False)


In [ ]:
df_PM

,SEQN,RXDUSE,RXDDRUG,RXDDRGID,RXQSEEN,RXDDAYS,RXDCOUNT,years,RXDRSC1,RXDRSC2,RXDRSC3,RXDRSD1,RXDRSD2,RXDRSD3
5,21010,1.0,ALBUTEROL,d00749,1.0,2190.0,3.0,0304,NaN,NaN,NaN,NaN,NaN,NaN
6,21010,1.0,CETIRIZINE,d03827,1.0,1095.0,3.0,0304,NaN,NaN,NaN,NaN,NaN,NaN
7,21010,1.0,IBUPROFEN,d00015,1.0,NaN,3.0,0304,NaN,NaN,NaN,NaN,NaN,NaN
9,21012,1.0,ACETAZOLAMIDE,d00161,1.0,5475.0,3.0,0304,NaN,NaN,NaN,NaN,NaN,NaN
10,21012,1.0,DORZOLAMIDE; TIMOLOL OPHTHALMIC,d04303,1.0,5475.0,3.0,0304,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32957,124821,1.0,MELOXICAM,d04532,1.0,2555.0,4.0,1720,M06.9,,,"Rheumatoid arthritis, unspecified",,
32958,124821,1.0,METOPROLOL,d00134,1.0,730.0,4.0,1720,I21.P,,,Prevent heart attack/myocardial infarction,,
32959,124821,1.0,TAMSULOSIN,d04121,1.0,3650.0,4.0,1720,N40,,,Enlarged prostate,,
32960,124822,1.0,ASPIRIN,d00170,2.0,1460.0,2.0,1720,I21.P,I63.P,,Prevent heart attack/myocardial infarction,Prevent stroke,


In [ ]:
df_PM_1 = df_PM.loc[(df_PM['RXDRSC1'] == 'F11.2') | (df_PM['RXDRSC1'] == 'F11.23')]

In [ ]:
drugs = pd.read_sas('../data/RXQ_DRUG.xpt', encoding='ISO-8859-1')
drug_60 = drugs[(drugs['RXDDCI1A'] == 57) & (drugs['RXDDCI1B'] == 58) & (drugs['RXDDCI1C'] == 60)]
drug_191 = drugs[(drugs['RXDDCI1A'] == 57) & (drugs['RXDDCI1B'] == 58) & (drugs['RXDDCI1C'] == 191)]
drug = pd.concat([drug_60, drug_191])
drug_id = set(drug['RXDDRGID'].tolist())

In [ ]:
df_PM  = df_PM [df_PM ['RXDDRGID'].isin(drug_id)]

In [ ]:
df_PM = pd.concat([df_PM, df_PM_1])
df_PM = df_PM.drop_duplicates()

In [ ]:
# 3992 records for taking opioid prescription or taking prescription for opioid dependence.
len(df_PM)

3992

In [ ]:
df_PM = df_PM[['SEQN', 'RXDDRUG', 'RXDDRGID', 'RXDDAYS']]
df_PM = df_PM[df_PM['RXDDAYS'] > 90]
df_PM[['SEQN', 'RXDDAYS']] = df_PM[['SEQN', 'RXDDAYS']].astype(int)

In [ ]:
"""
# We define the long term opioid users as those how have taken opioid prescriptions over 90 days.

# Note that each user can take multiple opioid prescriptions.

# In an earlier study for tracking long term opioid users,
# The author excluded medications containing buprenorphine since they are used to treat use disorder.
# However, we find multiple cases that this medicine used to treat opioid dependence.
# So technically this also implies the user is a long term opioid user.
"""
df_PM = df_PM.rename(columns={'RXDDRUG': 'drug_name', 'RXDDRGID': 'drug_id', 'RXDDAYS': 'days_using'})
df_PM = df_PM.set_index('SEQN')
df_PM

,drug_name,drug_id,days_using
SEQN,,,
21118,ACETAMINOPHEN; CODEINE,d03423,182
21165,ACETAMINOPHEN; HYDROCODONE,d03428,122
21178,ACETAMINOPHEN; HYDROCODONE,d03428,1825
21224,ACETAMINOPHEN; HYDROCODONE,d03428,730
21224,ACETAMINOPHEN; PROPOXYPHENE,d03434,1825
...,...,...,...
124500,ACETAMINOPHEN; HYDROCODONE,d03428,3285
124580,HYDROCODONE,d03075,1825
124615,OXYCODONE,d00329,365


#### Create labels for all users in the main table.

In [ ]:
opioid_user_set = set(df_PM.index.tolist())

In [ ]:
df_DU['active_user'].value_counts()

active_user
2    475
1    219
Name: count, dtype: int64

In [ ]:
"""
Following the labeling scheme, we get the label in the main table.
0: user, 1: active user, 2: recovered_user
"""
df_demo['label'] = 0
df_demo['label'] = df_demo.index.map(df_DU['active_user']).fillna(0)
df_demo.loc[df_demo.index.isin(opioid_user_set), 'label'] = 1

In [ ]:
df_demo['label'].value_counts()

label
0.0    92723
1.0     2728
2.0      421
Name: count, dtype: int64

In [ ]:
df_demo.to_csv('../processed_data/main_table.csv')

### 3. Laboratory Data

In [ ]:
type_laboratory = 'laboratory'

#### 3.1. Standard Biochemistry Profile

In [ ]:
df_SBP = concat_data_across_years(type_laboratory, 'BIOPRO',
                                 ['0506', '0708', '0910', '1112', '1314', '1516', '1718', '1720'], 'D')
df_temp = concat_data_across_years(type_laboratory, 'L40',
                                 ['0304'], 'C')
df_SBP = pd.concat([df_SBP, df_temp])

In [ ]:
# We keep all the items in this laboratory but filter out the same value with different unit.
# For example, we keep "Total calcium (mmol/L)" and get rid of "Total calcium (mg/dL)"
columns_SBP = [
    'SEQN',
    'LBDSALSI', 'LBXSATSI', 'LBXSASSI', 'LBXSAPSI', 'LBDSBUSI', 'LBDSCASI',
    'LBDSCHSI', 'LBXSC3SI', 'LBDSCRSI', 'LBXSGTSI', 'LBDSGLSI', 'LBDSIRSI',
    'LBXSLDSI', 'LBDSPHSI', 'LBDSTBSI', 'LBDSTPSI', 'LBDSTRSI', 'LBDSUASI',
    'LBDSCRSI', 'LBXSNASI', 'LBXSKSI', 'LBXSCLSI', 'LBXSOSSI', 'LBDSGBSI'
]
rename_SBP = {
    'LBDSALSI': 'Albumin (g/L)', 'LBXSATSI': 'Alanine aminotransferase (ALT) (U/L)',
    'LBXSASSI': 'Aspartate aminotransferase (AST) (U/L)', 'LBXSAPSI': 'Alkaline phosphatase (U/L)',
    'LBDSBUSI': 'Blood urea nitrogen (mmol/L)', 'LBDSCASI': 'Total calcium (mmol/L)',
    'LBDSCHSI': 'Cholesterol (mmol/L)', 'LBXSC3SI': 'Bicarbonate (mmol/L)',
    'LBDSCRSI': 'Creatinine (µmol/L)', 'LBXSGTSI': 'Gamma glutamyl transferase (U/L)',
    'LBDSGLSI': 'Glucose, serum (mmol/L)', 'LBDSIRSI': 'Iron, refrigerated (umol/L)',
    'LBXSLDSI': 'Lactate dehydrogenase LDH (U/L)', 'LBDSPHSI': 'Phosphorus (mmol/L)',
    'LBDSTBSI': 'Bilirubin, total (umol/L)', 'LBDSTPSI': 'Total protein (g/L)',
    'LBDSTRSI': 'Triglycerides (mmol/L)', 'LBDSUASI': 'Uric acid (umol/L)',
    'LBXSNASI': 'Sodium (mmol/L)',
    'LBXSKSI': 'Potassium (mmol/L)', 'LBXSCLSI': 'Chloride (mmol/L)',
    'LBXSOSSI': 'Osmolality (mmol/Kg)', 'LBDSGBSI': 'Globulin (g/L)'
}

In [ ]:
# Not every respondent take this examination. In the 95872 respondents from 2003-2020, we have 60331 valid records.
df_SBP = df_SBP[columns_SBP]
df_SBP = df_SBP.rename(columns=rename_SBP)
df_SBP['SEQN'] = df_SBP['SEQN'].astype(int)
df_SBP = df_SBP.set_index('SEQN')
df_SBP.dropna(how='all', inplace=True)
df_SBP

,Albumin (g/L),Alanine aminotransferase (ALT) (U/L),Aspartate aminotransferase (AST) (U/L),Alkaline phosphatase (U/L),Blood urea nitrogen (mmol/L),Total calcium (mmol/L),Cholesterol (mmol/L),Bicarbonate (mmol/L),Creatinine (µmol/L),Gamma glutamyl transferase (U/L),...,"Bilirubin, total (umol/L)",Total protein (g/L),Triglycerides (mmol/L),Uric acid (umol/L),Creatinine (µmol/L),Sodium (mmol/L),Potassium (mmol/L),Chloride (mmol/L),Osmolality (mmol/Kg),Globulin (g/L)
SEQN,,,,,,,,,,,,,,,,,,,,,
31129,43.0,25.0,25.0,88.0,3.57,2.425,4.396,24.0,88.40,34.0,...,10.26,70.0,0.565,398.5,88.40,141.0,4.2,106.0,279.0,27.0
31131,35.0,14.0,16.0,74.0,2.14,2.225,2.664,23.0,70.72,17.0,...,6.84,69.0,0.881,291.5,70.72,137.0,4.1,106.0,271.0,34.0
31132,50.0,31.0,29.0,48.0,8.93,2.475,3.931,29.0,106.08,22.0,...,17.10,72.0,0.666,428.3,106.08,140.0,3.8,102.0,287.0,22.0
31133,42.0,15.0,21.0,41.0,2.50,2.450,3.853,23.0,79.56,27.0,...,30.78,78.0,0.553,279.6,79.56,137.0,4.0,102.0,271.0,36.0
31134,39.0,30.0,31.0,77.0,4.64,2.375,4.784,27.0,106.08,33.0,...,8.55,71.0,2.055,446.1,106.08,139.0,4.1,103.0,277.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31119,45.0,97.0,64.0,92.0,3.57,2.475,5.689,23.0,70.72,91.0,...,11.97,73.0,2.258,333.1,70.72,137.0,3.9,102.0,274.0,29.0
31121,39.0,20.0,31.0,72.0,3.93,2.300,5.353,26.0,70.72,29.0,...,8.55,73.0,1.716,226.0,70.72,140.0,3.6,101.0,279.0,34.0
31123,44.0,19.0,27.0,37.0,5.36,2.400,6.982,26.0,79.56,7.0,...,11.97,70.0,0.768,273.6,79.56,140.0,4.1,104.0,280.0,27.0


In [ ]:
# 89% of the active users have this data; 93% of the recovered users have this data.
check_coverage(df_demo, df_SBP)

KeyError: "None of [Index(['label'], dtype='object')] are in the [columns]"

#### 3.2 Complete Blood Count

In [ ]:
df_CBC = concat_data_across_years(type_laboratory, 'CBC',
                                 ['0506', '0708', '0910', '1112', '1314', '1516', '1718', '1720'], 'D')
df_temp = concat_data_across_years(type_laboratory, 'L25',
                                 ['0304'], 'C')
df_CBC = pd.concat([df_CBC, df_temp])

In [ ]:
columns_CBC = [
    'SEQN',
    'LBXWBCSI', 'LBXLYPCT', 'LBXMOPCT', 'LBXNEPCT', 'LBXEOPCT',
    'LBXBAPCT', 'LBDLYMNO', 'LBDMONO', 'LBDNENO', 'LBDEONO',
    'LBDBANO', 'LBXRBCSI', 'LBXHGB', 'LBXHCT', 'LBXMCVSI',
    'LBXMCHSI', 'LBXMC', 'LBXRDW', 'LBXPLTSI', 'LBXMPSI'
]
rename_CBC = {
    'LBXWBCSI': 'White blood cell count (1000 cells/uL)',
    'LBXLYPCT': 'Lymphocyte percent (%)',
    'LBXMOPCT': 'Monocyte percent (%)',
    'LBXNEPCT': 'Segmented neutrophils percent (%)',
    'LBXEOPCT': 'Eosinophils percent (%)',
    'LBXBAPCT': 'Basophils percent (%)',
    'LBDLYMNO': 'Lymphocyte number (1000 cells/uL)',
    'LBDMONO': 'Monocyte number (1000 cells/uL)',
    'LBDNENO': 'Segmented neutrophils num (1000 cell/uL)',
    'LBDEONO': 'Eosinophils number (1000 cells/uL)',
    'LBDBANO': 'Basophils number (1000 cells/uL)',
    'LBXRBCSI': 'Red blood cell count (million cells/uL)',
    'LBXHGB': 'Hemoglobin (g/dL)',
    'LBXHCT': 'Hematocrit (%)',
    'LBXMCVSI': 'Mean cell volume (fL)',
    'LBXMCHSI': 'Mean cell hemoglobin (pg)',
    'LBXMC': 'Mean Cell Hgb Conc. (g/dL)',
    'LBXRDW': 'Red cell distribution width (%)',
    'LBXPLTSI': 'Platelet count (1000 cells/uL)',
    'LBXMPSI': 'Mean platelet volume (fL)'
}

In [ ]:
df_CBC = df_CBC[columns_CBC]
df_CBC = df_CBC.rename(columns=rename_CBC)
df_CBC['SEQN'] = df_CBC['SEQN'].astype(int)
df_CBC = df_CBC.set_index('SEQN')
df_CBC.dropna(how='all', inplace=True)
df_CBC

,White blood cell count (1000 cells/uL),Lymphocyte percent (%),Monocyte percent (%),Segmented neutrophils percent (%),Eosinophils percent (%),Basophils percent (%),Lymphocyte number (1000 cells/uL),Monocyte number (1000 cells/uL),Segmented neutrophils num (1000 cell/uL),Eosinophils number (1000 cells/uL),Basophils number (1000 cells/uL),Red blood cell count (million cells/uL),Hemoglobin (g/dL),Hematocrit (%),Mean cell volume (fL),Mean cell hemoglobin (pg),Mean Cell Hgb Conc. (g/dL),Red cell distribution width (%),Platelet count (1000 cells/uL),Mean platelet volume (fL)
SEQN,,,,,,,,,,,,,,,,,,,,
31128,5.0,45.3,8.6,44.3,1.8,0.1,2.3,0.4,2.2,0.1,0.0,5.25,13.7,41.4,78.8,26.1,33.1,12.6,286.0,8.1
31129,8.2,15.2,12.7,59.9,11.9,0.3,1.2,1.0,4.9,1.0,0.0,4.78,14.1,41.5,86.9,29.5,34.0,12.4,214.0,8.9
31131,5.3,35.8,7.8,55.1,0.9,0.5,1.9,0.4,2.9,0.0,0.0,4.63,12.5,37.1,80.1,27.1,33.8,13.7,298.0,7.8
31132,7.5,29.4,9.1,58.9,2.2,0.4,2.2,0.7,4.4,0.2,0.0,4.72,14.5,42.6,90.3,30.7,34.0,12.5,225.0,8.6
31133,6.6,30.0,5.7,63.1,1.2,0.1,2.0,0.4,4.2,0.1,0.0,3.86,12.9,38.3,99.0,33.3,33.7,14.5,226.0,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31121,8.6,29.2,9.4,57.7,3.4,0.4,2.5,0.8,5.0,0.3,0.0,4.03,10.4,31.9,79.2,25.9,32.8,14.3,474.0,6.8
31123,5.4,32.2,10.2,55.4,1.8,0.4,1.7,0.6,3.0,0.1,0.0,3.93,12.4,36.5,92.8,31.5,34.0,13.0,271.0,7.5
31124,6.9,38.8,10.2,48.7,1.7,0.6,2.7,0.7,3.4,0.1,0.0,5.47,17.0,50.1,91.4,30.9,33.8,12.9,268.0,8.6


In [ ]:
check_coverage(df_demo, df_CBC)

label
0.0    81.19
1.0    91.20
2.0    96.20
Name: count, dtype: float64

In [ ]:
df_medical = pd.merge(df_SBP, df_CBC, left_index=True, right_index=True, how='outer')

In [ ]:
df_medical.to_csv('../processed_data/medical_table.csv')

### 4. Dietary Data

In [ ]:
type_dietary = 'dietary'

#### 4.1. Total Nutrition Intakes

In [ ]:
df_TOT1 = concat_data_across_years(type_dietary, 'DR1TOT', years, year_char)
df_TOT2 = concat_data_across_years(type_dietary, 'DR2TOT', years, year_char)

In [ ]:
nutrition_facts = [
    "SEQN",
    "DBQ095Z", "DBD100", "DRQSPREP", "DR1STY", "DR1SKY", "DRQSDIET",
    "DRQSDT1", "DRQSDT2", "DRQSDT3", "DRQSDT4", "DRQSDT5", "DRQSDT6",
    "DRQSDT7", "DRQSDT8", "DRQSDT9", "DRQSDT10", "DRQSDT11", "DRQSDT12",
    "DRQSDT91", "DR1TNUMF", "DR1TWSZ", "DR1_300", "DRD340", "DRD360"
]
nutrition_data_1 = [
    "SEQN",
    "DR1TKCAL", "DR1TPROT", "DR1TCARB", "DR1TSUGR", "DR1TFIBE", "DR1TTFAT",
    "DR1TSFAT", "DR1TMFAT", "DR1TPFAT", "DR1TCHOL", "DR1TATOC", "DR1TATOA",
    "DR1TRET", "DR1TVARA", "DR1TACAR", "DR1TBCAR", "DR1TCRYP", "DR1TLYCO",
    "DR1TLZ", "DR1TVB1", "DR1TVB2", "DR1TNIAC", "DR1TVB6", "DR1TFOLA",
    "DR1TFA", "DR1TFF", "DR1TFDFE", "DR1TCHL", "DR1TVB12", "DR1TB12A",
    "DR1TVC", "DR1TVD", "DR1TVK", "DR1TCALC", "DR1TPHOS", "DR1TMAGN",
    "DR1TIRON", "DR1TZINC", "DR1TCOPP", "DR1TSODI", "DR1TPOTA", "DR1TSELE",
    "DR1TCAFF", "DR1TTHEO", "DR1TALCO", "DR1TMOIS", "DR1_320Z", "DR1_330Z",
    "DR1BWATZ"
]
nutrition_data_2 = [
    "SEQN",
    "DR2TKCAL", "DR2TPROT",
    "DR2TCARB", "DR2TSUGR", "DR2TFIBE", "DR2TTFAT", "DR2TSFAT", "DR2TMFAT",
    "DR2TPFAT", "DR2TCHOL", "DR2TATOC", "DR2TATOA", "DR2TRET", "DR2TVARA",
    "DR2TACAR", "DR2TBCAR", "DR2TCRYP", "DR2TLYCO", "DR2TLZ", "DR2TVB1",
    "DR2TVB2", "DR2TNIAC", "DR2TVB6", "DR2TFOLA", "DR2TFA", "DR2TFF",
    "DR2TFDFE", "DR2TCHL", "DR2TVB12", "DR2TB12A", "DR2TVC", "DR2TVD",
    "DR2TVK", "DR2TCALC", "DR2TPHOS", "DR2TMAGN", "DR2TIRON", "DR2TZINC",
    "DR2TCOPP", "DR2TSODI", "DR2TPOTA", "DR2TSELE", "DR2TCAFF", "DR2TTHEO",
    "DR2TALCO", "DR2TMOIS", "DR2_320Z", "DR2_330Z", "DR2BWATZ"
]

In [ ]:
df_NF = df_TOT1[nutrition_facts]
df_ND1 = df_TOT1[nutrition_data_1]
df_ND2 = df_TOT2[nutrition_data_2]

In [ ]:
df_ND = pd.DataFrame(np.vstack((df_ND1.to_numpy(), df_ND2.to_numpy())), columns=df_ND1.columns)
df_ND = df_ND.groupby('SEQN').mean().reset_index().fillna(0)

In [ ]:
df_ND = df_ND.set_index('SEQN')
standard_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()
df_standardized = pd.DataFrame(standard_scaler.fit_transform(df_ND), columns=df_ND.columns, index=df_ND.index)
df_ND = pd.DataFrame(minmax_scaler.fit_transform(df_standardized), columns=df_ND.columns, index=df_ND.index)

In [ ]:
df_NF = df_NF.fillna(-1).astype(int)
df_NF = df_NF.set_index('SEQN')
df_N = pd.merge(df_ND, df_NF, left_index=True, right_index=True, how='outer')
df_N.index.name = 'SEQN'

In [ ]:
df_N

,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,...,DRQSDT9,DRQSDT10,DRQSDT11,DRQSDT12,DRQSDT91,DR1TNUMF,DR1TWSZ,DR1_300,DRD340,DRD360
SEQN,,,,,,,,,,,,,,,,,,,,,
21005.0,0.192279,0.148351,0.211728,0.178718,0.091757,0.161858,0.118139,0.150943,0.126867,0.092378,...,-1,-1,-1,-1,-1,17,-1,2,-1,-1
21006.0,0.088682,0.058510,0.106858,0.066880,0.061431,0.068764,0.057119,0.080049,0.029753,0.097153,...,-1,-1,-1,-1,-1,10,-1,3,2,1
21007.0,0.069805,0.047795,0.082592,0.060816,0.043157,0.055477,0.043491,0.060798,0.028918,0.029568,...,-1,-1,-1,-1,-1,10,-1,2,-1,-1
21008.0,0.165482,0.136436,0.166369,0.098965,0.082426,0.151714,0.133586,0.160419,0.057762,0.150046,...,-1,-1,-1,-1,-1,4,-1,2,-1,-1
21009.0,0.336109,0.205349,0.366953,0.327535,0.148911,0.317571,0.244863,0.330901,0.200658,0.206428,...,-1,-1,-1,-1,-1,22,-1,2,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124818.0,0.251462,0.149473,0.262234,0.223684,0.145023,0.253629,0.233705,0.215902,0.153561,0.100643,...,-1,-1,-1,-1,-1,9,4,3,1,1
124819.0,0.107040,0.080404,0.105534,0.074706,0.070762,0.106140,0.098004,0.098844,0.051776,0.043893,...,-1,-1,-1,-1,-1,14,1,2,2,2
124820.0,0.106077,0.075811,0.111332,0.070986,0.086314,0.101001,0.083250,0.101335,0.053845,0.046465,...,-1,-1,-1,-1,-1,8,91,2,2,2


In [ ]:
df_N.to_csv('../processed_data/diet_table.csv')

In [ ]:
check_coverage(df_demo, df_N)

label
0.0     97.99
1.0     98.31
2.0    100.00
Name: count, dtype: float64

### Miscellaneous Analysis - Broken below

In [ ]:
df_IFF1 = concat_data_across_years(type_dietary, 'DR1IFF', years, year_char)
df_IFF2 = concat_data_across_years(type_dietary, 'DR2IFF', years, year_char)

In [ ]:
# See here that not all our target has detailed food records. And this rate is 88.45%
df_temp = df_IFF1.set_index('SEQN')
check_coverage(df_demo, df_temp)

label
0.0    86.85
1.0    88.45
2.0    96.44
Name: count, dtype: float64

In [ ]:
# df_IFF = pd.concat([df_IFF1, df_IFF2])
# df_IFF = df_IFF.groupby('SEQN').count().reset_index()
# temp = df_demo.merge(df_IFF, on='SEQN', how='left')
# temp = temp[['SEQN', 'label', 'Food code']]
# temp.groupby('label')['Food code'].mean()